In [ ]:
import os
import warnings
warnings.filterwarnings("ignore") 

In [ ]:
# .wav files which were converted from .flac
os.listdir("./wavfiles/audio/")

['1089-134686-0000.wav',
 '1089-134686-0001.wav',
 '1089-134686-0002.wav',
 '1089-134686-0003.wav',
 '1089-134686-0004.wav',
 '1089-134686-0005.wav',
 '1089-134686-0006.wav',
 '1089-134686-0007.wav',
 '1089-134686-0008.wav',
 '1089-134686-0009.wav',
 '1089-134686-0010.wav',
 '1089-134686-0011.wav',
 '1089-134686-0012.wav',
 '1089-134686-0013.wav',
 '1089-134686-0014.wav',
 '1089-134686-0015.wav',
 '1089-134686-0016.wav',
 '1089-134686-0017.wav',
 '1089-134686-0018.wav',
 '1089-134686-0019.wav',
 '1089-134686-0020.wav',
 '1089-134686-0021.wav',
 '1089-134686-0022.wav',
 '1089-134686-0023.wav',
 '1089-134686-0024.wav',
 '1089-134686-0025.wav',
 '1089-134686-0026.wav',
 '1089-134686-0027.wav',
 '1089-134686-0028.wav',
 '1089-134686-0029.wav',
 '1089-134686-0030.wav',
 '1089-134686-0031.wav',
 '1089-134686-0032.wav',
 '1089-134686-0033.wav',
 '1089-134686-0034.wav',
 '1089-134686-0035.wav',
 '1089-134686-0036.wav',
 '1089-134686-0037.wav',
 '1089-134691-0000.wav',
 '1089-134691-0001.wav',


In [ ]:
# The ASR repo in NeMo has the py file which uses jasper to produce transcriptions for the audio files.
# It gives a json file with the transcriptions as output. 
%%time
!python ./NeMo-main/examples/asr/transcribe_speech.py pretrained_name="stt_en_jasper10x5dr" audio_dir="./wavfiles/audio/"

[NeMo W 2021-12-14 17:32:41 optimizers:50] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo I 2021-12-14 17:32:45 transcribe_speech:74] Hydra config: model_path: null
    pretrained_name: stt_en_jasper10x5dr
    audio_dir: ./wavfiles/audio/
    dataset_manifest: null
    output_filename: null
    batch_size: 32
    cuda: null
    amp: false
    audio_type: wav
    rnnt_decoding:
      strategy: greedy_batch
      compute_hypothesis_token_set: false
      greedy:
        max_symbols_per_step: null
        preserve_alignments: false
      beam:
        beam_siz

Extraction of transcription from the json file

In [ ]:
import json 

In [ ]:
transcription = []
for line in open('./wavfiles/audio.json', 'r'):
    transcription.append(json.loads(line))

In [ ]:
transcription

[{'audio_filepath': './wavfiles/audio/5105-28241-0018.wav',
  'pred_text': 'its depth remained invariable still for or at most five fathoms and although its bottom was assiduously dredged it was only to prove it barren of marine production of any type'},
 {'audio_filepath': './wavfiles/audio/4507-16021-0012.wav',
  'pred_text': 'why should one halt on the way'},
 {'audio_filepath': './wavfiles/audio/672-122797-0059.wav',
  'pred_text': 'only that one answered the tree'},
 {'audio_filepath': './wavfiles/audio/7021-79740-0003.wav',
  'pred_text': 'to give an idea of these conversations i will report one of them in full'},
 {'audio_filepath': './wavfiles/audio/8230-279154-0010.wav',
  'pred_text': "we sometimes have images that are by no means peculiarly vague which yet we do not trust for example under the influence of fatigue we may see a france's face vividly and clearly but horribly distorted"},
 {'audio_filepath': './wavfiles/audio/2830-3980-0000.wav',
  'pred_text': 'in every way th

In [ ]:
#import WER from dataset library
from datasets import load_metric

metric = load_metric("wer")

In [ ]:
!unzip -qq "/content/drive/MyDrive/Jasper/text.zip"

In [ ]:
#Storing the real transcrption from the txt files one by one
import os
lines = []
for i in os.listdir("./text"):
    file1 = open("./text/"+i)
    for f in file1:
        lines.append(f)
    file1.close()

In [ ]:
#Separating the audio file names and the corresponding transcriptions
aud_file = []
trans_data = []
for i in lines:
    aud = i.split(" ")[0]
    tran = i.split(" ")[1:]
    aud_file.append(aud.strip())
    trans_data.append(" ".join(tran).strip())

In [ ]:
#converting it to a dataframe for easy access
import pandas as pd
trans_df = pd.DataFrame({'File_name':aud_file,'Transcription':trans_data})
trans_df.set_index("File_name",inplace=True)

In [ ]:
#Number of transcriptions in the available (should be equal to the number of speech files) 
len(trans_df)

2620

In [ ]:
trans_df.head()

,Transcription
File_name,
8555-292519-0000,BRIGHTER THAN EARLY DAWN'S MOST BRILLIANT DYE ...
8555-292519-0001,GUIDED BY YOU HOW WE MIGHT STROLL TOWARDS DEAT...
8555-292519-0002,VENICE
8555-292519-0003,IN A SUNSET GLOWING OF CRIMSON AND GOLD SHE LI...
8555-292519-0004,THE PITY THAT WE MUST COME AND GO


In [ ]:
#Calculating the WER for the dataset
scores = 0
pred_txts = [] 
for i in transcription:
    trans_pred = i['pred_text']
    pred_txts.append(trans_pred)
    aud_f = i["audio_filepath"].split('/')[-1].split('.')[0]
    trans_act = trans_df.loc[aud_f]["Transcription"]
    score = metric.compute(predictions=[trans_pred], references=[trans_act.lower()])
    scores = score + scores

In [ ]:
print("WER for the  Librispeech test clean dataset is:",scores/len(pred_txts)*100)

WER for the  Librispeech test clean dataset is: 4.101566397781093


## WER for the test-clean dataset using Jasper is 4.10%